In [23]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [24]:
out='../output/'
inp='../input/monthly/'
pref='dam'

In [25]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
264,(2021/07/11) 23-24,57.94,20
265,Min,57.94,0
266,Max,200.79,60
267,Average,93.93253787878787878787878788,19.090909090909090909090909091


In [26]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/03/06) 04-05,29.00,50.0
6,(2017/03/06) 05-06,29.00,50.0
7,(2017/03/06) 06-07,65.00,0.0
8,(2017/03/06) 07-08,97.21,0.0
9,(2017/03/06) 08-09,97.21,0.0
...,...,...,...
260,(2021/07/11) 19-20,57.94,20.0
261,(2021/07/11) 20-21,57.94,20.0
262,(2021/07/11) 21-22,57.94,20.0
263,(2021/07/11) 22-23,57.94,20.0


In [27]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/03/06) 04-05,29.00,50.0,2017-03-06,4
6,(2017/03/06) 05-06,29.00,50.0,2017-03-06,5
7,(2017/03/06) 06-07,65.00,0.0,2017-03-06,6
8,(2017/03/06) 07-08,97.21,0.0,2017-03-06,7
9,(2017/03/06) 08-09,97.21,0.0,2017-03-06,8
...,...,...,...,...,...
260,(2021/07/11) 19-20,57.94,20.0,2021-07-11,19
261,(2021/07/11) 20-21,57.94,20.0,2021-07-11,20
262,(2021/07/11) 21-22,57.94,20.0,2021-07-11,21
263,(2021/07/11) 22-23,57.94,20.0,2021-07-11,22


In [28]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
5,29.00,50.0,2017-03-06,4
6,29.00,50.0,2017-03-06,5
7,65.00,0.0,2017-03-06,6
8,97.21,0.0,2017-03-06,7
9,97.21,0.0,2017-03-06,8
...,...,...,...,...
260,57.94,20.0,2021-07-11,19
261,57.94,20.0,2021-07-11,20
262,57.94,20.0,2021-07-11,21
263,57.94,20.0,2021-07-11,22


In [29]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0
2017-03-06,29.00,50.0,5,2017,3,6,0
2017-03-06,65.00,0.0,6,2017,3,6,0
2017-03-06,97.21,0.0,7,2017,3,6,0
2017-03-06,97.21,0.0,8,2017,3,6,0
...,...,...,...,...,...,...,...
2021-07-11,57.94,20.0,19,2021,7,11,6
2021-07-11,57.94,20.0,20,2021,7,11,6
2021-07-11,57.94,20.0,21,2021,7,11,6


In [30]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.groupby(['year','month','day']).sum().drop(['hour'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()

daily

volume  day_of_week    value  price_weight_av  price_max  \
year month day                                                             
2017 3     6     200.0            0   5800.0            29.00      97.21   
           7     400.0           24  11600.0            29.00      97.21   
           8     400.0           48  11600.0            29.00      97.21   
           9     400.0           72  11600.0            29.00      97.21   
           10    200.0           80   5800.0            29.00      97.21   
...                ...          ...      ...              ...        ...   
2021 7     7     160.0           48   9270.4            57.94     200.00   
           8     160.0           72   9270.4            57.94     200.00   
           9     160.0           96   9270.4            57.94     200.00   
           10    340.0          120  19699.6            57.94     107.00   
           11    480.0          144  27811.2            57.94      57.94   

                price_min  price_mean  power_max  power_min  power_mean  \
year month day                                                            
2017 3     6        29.00   65.852500       50.0        0.0   10.000000   
           7        29.00   59.710417       50.0        0.0   16.666667   
           8        29.00   59.710417       50.0        0.0   16.666667   
           9        29.00   59.710417       50.0        0.0   16.666667   
           10       29.00   65.852500       50.0        0.0   10.000000   
...                   ...         ...        ...        ...         ...   
2021 7     7        57.94  110.021667       20.0        0.0    6.666667   
           8        57.94  110.021667       20.0        0.0    6.666667   
           9        57.94  110.021667       20.0        0.0    6.666667   
           10       57.94   72.249167       20.0        0.0   14.166667   
           11       57.94   57.940000       20.0       20.0   20.000000   

                price_morning_max  price_evening_max  price_standard  
year month day                                                        
2017 3     6                97.21              97.21           65.00  
           7                97.21              97.21           65.00  
           8                97.21              97.21           65.00  
           9                97.21              97.21           65.00  
           10               97.21              97.21           65.00  
...                           ...                ...             ...  
2021 7     7               200.00             200.00          107.00  
           8               200.00             200.00          107.00  
           9               200.00             200.00          107.00  
           10              107.00             107.00           57.94  
           11               57.94              57.94           57.94  

[1535 rows x 13 columns]

In [31]:
hourly['cat']='off'

if (pref=='dam'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0,1450.0,off
2017-03-06,29.00,50.0,5,2017,3,6,0,1450.0,standard
2017-03-06,65.00,0.0,6,2017,3,6,0,0.0,morning
2017-03-06,97.21,0.0,7,2017,3,6,0,0.0,morning
2017-03-06,97.21,0.0,8,2017,3,6,0,0.0,morning
...,...,...,...,...,...,...,...,...,...
2021-07-11,57.94,20.0,19,2021,7,11,6,1158.8,off
2021-07-11,57.94,20.0,20,2021,7,11,6,1158.8,off
2021-07-11,57.94,20.0,21,2021,7,11,6,1158.8,off


In [32]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume  day_of_week    value  price_weight_av  price_max  \
year month day                                                             
2017 3     6     200.0            0   5800.0            29.00      97.21   
           7     400.0           24  11600.0            29.00      97.21   
           8     400.0           48  11600.0            29.00      97.21   
           9     400.0           72  11600.0            29.00      97.21   
           10    200.0           80   5800.0            29.00      97.21   
...                ...          ...      ...              ...        ...   
2021 7     7     160.0           48   9270.4            57.94     200.00   
           8     160.0           72   9270.4            57.94     200.00   
           9     160.0           96   9270.4            57.94     200.00   
           10    340.0          120  19699.6            57.94     107.00   
           11    480.0          144  27811.2            57.94      57.94   

                price_min  price_mean  power_max  power_min  power_mean  ...  \
year month day                                                           ...   
2017 3     6        29.00   65.852500       50.0        0.0   10.000000  ...   
           7        29.00   59.710417       50.0        0.0   16.666667  ...   
           8        29.00   59.710417       50.0        0.0   16.666667  ...   
           9        29.00   59.710417       50.0        0.0   16.666667  ...   
           10       29.00   65.852500       50.0        0.0   10.000000  ...   
...                   ...         ...        ...        ...         ...  ...   
2021 7     7        57.94  110.021667       20.0        0.0    6.666667  ...   
           8        57.94  110.021667       20.0        0.0    6.666667  ...   
           9        57.94  110.021667       20.0        0.0    6.666667  ...   
           10       57.94   72.249167       20.0        0.0   14.166667  ...   
           11       57.94   57.940000       20.0       20.0   20.000000  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2017 3     6                   3.0                 2.0       150.0   
           7                   3.0                 2.0       350.0   
           8                   3.0                 2.0       350.0   
           9                   3.0                 2.0       350.0   
           10                  3.0                 2.0       150.0   
...                            ...                 ...         ...   
2021 7     7                   3.0                 2.0       140.0   
           8                   3.0                 2.0       140.0   
           9                   3.0                 2.0       140.0   
           10                  NaN                 NaN       340.0   
           11                  NaN                 NaN       480.0   

                standard_volume  morning_volume  evening_volume  off_value  \
year month day                                                               
2017 3     6               50.0             0.0             0.0     4350.0   
           7               50.0             0.0             0.0    10150.0   
           8               50.0             0.0             0.0    10150.0   
           9               50.0             0.0             0.0    10150.0   
           10              50.0             0.0             0.0     4350.0   
...                         ...             ...             ...        ...   
2021 7     7               20.0             0.0             0.0     8111.6   
           8               20.0             0.0             0.0     8111.6   
           9               20.0             0.0             0.0     8111.6   
           10               0.0             NaN             NaN    19699.6   
           11               NaN             NaN             NaN    27811.2   

                standard_value  morning_value  evening_value

In [33]:
daily.to_csv(out + pref + '_daily.csv')

In [34]:
hourly.to_csv(out + pref + '_hourly.csv')

In [35]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat.loc[(hourly_flat['year']<2021)][['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2017-03-06,4,29.00,50.0,off
2017-03-06,5,29.00,50.0,standard
2017-03-06,6,65.00,0.0,morning
2017-03-06,7,97.21,0.0,morning
2017-03-06,8,97.21,0.0,morning
...,...,...,...,...
2020-12-31,19,90.00,0.0,evening
2020-12-31,20,84.95,20.0,off
2020-12-31,21,84.95,20.0,off


In [36]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [37]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,day_of_week,value_mean,price_max,price_min,price_mean,power_max,power_min,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,591.315789,64.000000,18286.034211,65.852500,29.00,50.245965,50.000000,9.473684,...,30.0,30.924312,27.471153,34.148967,64.340852,86.158317,41.902439,7.822222,12.750000,6.733333
1,2017,4,1148.500000,67.000000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,...,4.0,33.403017,24.330980,41.851838,76.297844,87.970000,59.745455,44.533333,30.874074,51.800000
2,2017,5,949.871429,69.523810,42251.110429,57.075833,23.54,42.827163,61.290476,30.352381,...,30.0,44.480873,26.885423,47.257466,79.193597,88.723370,41.300000,50.655556,34.235088,60.633333
3,2017,6,477.653846,65.384615,23265.706154,71.232500,29.95,58.297673,28.461538,10.423077,...,40.0,48.708299,33.990683,61.625498,86.514416,115.321589,24.819788,11.700000,19.868778,7.550000
4,2017,7,460.096774,69.548387,20461.909516,70.355000,25.05,52.803882,22.838710,11.580645,...,42.0,44.473056,32.110278,59.895533,79.061769,110.973696,24.547486,10.317460,18.755102,5.476190
5,2017,8,455.000000,65.806452,22275.962903,64.280500,28.61,52.013892,23.419355,14.354839,...,46.0,48.958160,32.666806,57.921955,82.467739,99.018571,21.492537,16.405797,19.771318,14.608696
6,2017,9,434.916667,71.333333,19719.496667,63.606500,26.93,50.928794,21.000000,12.583333,...,42.0,45.340862,31.515333,57.501670,75.761157,94.614187,21.301775,12.619048,19.091837,8.928571
7,2017,10,634.529032,66.193548,29852.260903,60.922500,26.92,49.062565,29.032258,22.554839,...,44.0,47.046328,31.385187,58.422616,76.447836,88.679545,29.348571,22.969697,28.235484,20.000000
8,2017,11,657.756667,68.400000,28031.525633,59.438500,26.14,48.175603,38.596667,22.306667,...,44.0,42.616863,28.747793,55.714666,76.915099,85.168890,35.936970,22.227273,21.691532,23.340909
9,2017,12,621.906452,72.903226,28654.759419,64.220000,26.61,49.656720,32.809677,20.435484,...,42.0,46.075675,31.034572,59.088798,78.054909,89.241905,30.109497,21.920635,25.628163,20.000000


In [38]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,day_of_week,value_mean,price_max,price_min,price_mean,power_max,power_min,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,591.315789,64.000000,18286.034211,65.852500,29.00,50.245965,50.000000,9.473684,...,30.0,30.924312,27.471153,34.148967,64.340852,86.158317,41.902439,7.822222,12.750000,6.733333
1,2017,4,1148.500000,67.000000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,...,4.0,33.403017,24.330980,41.851838,76.297844,87.970000,59.745455,44.533333,30.874074,51.800000
2,2017,5,949.871429,69.523810,42251.110429,57.075833,23.54,42.827163,61.290476,30.352381,...,30.0,44.480873,26.885423,47.257466,79.193597,88.723370,41.300000,50.655556,34.235088,60.633333
3,2017,6,477.653846,65.384615,23265.706154,71.232500,29.95,58.297673,28.461538,10.423077,...,40.0,48.708299,33.990683,61.625498,86.514416,115.321589,24.819788,11.700000,19.868778,7.550000
4,2017,7,460.096774,69.548387,20461.909516,70.355000,25.05,52.803882,22.838710,11.580645,...,42.0,44.473056,32.110278,59.895533,79.061769,110.973696,24.547486,10.317460,18.755102,5.476190
5,2017,8,455.000000,65.806452,22275.962903,64.280500,28.61,52.013892,23.419355,14.354839,...,46.0,48.958160,32.666806,57.921955,82.467739,99.018571,21.492537,16.405797,19.771318,14.608696
6,2017,9,434.916667,71.333333,19719.496667,63.606500,26.93,50.928794,21.000000,12.583333,...,42.0,45.340862,31.515333,57.501670,75.761157,94.614187,21.301775,12.619048,19.091837,8.928571
7,2017,10,634.529032,66.193548,29852.260903,60.922500,26.92,49.062565,29.032258,22.554839,...,44.0,47.046328,31.385187,58.422616,76.447836,88.679545,29.348571,22.969697,28.235484,20.000000
8,2017,11,657.756667,68.400000,28031.525633,59.438500,26.14,48.175603,38.596667,22.306667,...,44.0,42.616863,28.747793,55.714666,76.915099,85.168890,35.936970,22.227273,21.691532,23.340909
9,2017,12,621.906452,72.903226,28654.759419,64.220000,26.61,49.656720,32.809677,20.435484,...,42.0,46.075675,31.034572,59.088798,78.054909,89.241905,30.109497,21.920635,25.628163,20.000000


In [39]:
monthly.to_csv(out + pref + '_monthly.csv')

In [40]:
monthly_filt=monthly.loc[monthly['year']<2020]
daily_filt=daily.reset_index().loc[daily.reset_index()['year']<2020]

In [41]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,day_of_week,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,volume,off_volume,standard_volume,morning_volume,evening_volume,value,off_power,standard_power,morning_power,evening_power
0,1,65.225806,3761.304500,40.800390,95.004839,27.899355,53.293397,5.358065,2.546774,3.706976,...,2644.150000,1664.800000,648.350000,190.200000,140.800000,1.166004e+05,4.969552,2.512984,2.756522,3.060870
1,2,70.530612,2417.153388,56.560000,102.117347,28.066122,66.335517,9.685714,0.000000,2.876888,...,1627.100000,1295.500000,321.350000,10.250000,0.000000,5.922026e+04,5.353306,1.879240,0.227778,0.000000
2,3,70.370370,8248.238889,27.459789,76.193457,26.825802,49.883502,21.814815,4.456790,11.224794,...,7190.666667,6157.333333,848.666667,117.333333,67.333333,2.227024e+05,23.480094,4.853041,2.607407,2.244444
3,4,66.625000,10005.681406,30.693082,68.357812,27.090313,46.932716,20.775000,4.718750,11.520964,...,5536.666667,3800.000000,1072.533333,372.733333,291.400000,2.134545e+05,27.955204,13.517402,19.153247,22.330519
4,5,67.855422,15561.203843,38.256259,74.521807,29.157349,49.178425,25.832530,8.522892,14.577972,...,9398.433333,5033.533333,2563.733333,994.166667,807.000000,4.305266e+05,18.847264,13.785081,20.281320,24.573430
5,6,69.720930,19682.468093,42.443435,113.796512,32.144884,64.629375,37.974419,7.417442,19.696831,...,12818.866667,10413.200000,2062.333333,238.333333,105.000000,5.642308e+05,31.219698,9.162252,3.968783,2.619841
6,7,66.795699,21168.929806,45.710761,125.730538,35.008387,71.307082,36.073118,7.515054,19.388629,...,13861.933333,10909.600000,2241.000000,425.666667,285.666667,6.562368e+05,31.807991,9.037157,6.605820,6.575397
7,8,68.774194,21616.049935,55.169621,131.696882,35.502688,71.295801,27.655914,7.647312,17.289830,...,12282.133333,7799.333333,3128.466667,799.000000,555.333333,6.700975e+05,23.130232,12.171248,11.639218,12.072464
8,9,70.311111,33145.984444,52.037704,92.956000,30.823556,54.966431,54.683333,12.066667,30.263565,...,20885.833333,15020.000000,4275.833333,951.666667,638.333333,9.943795e+05,43.615488,17.810721,15.535185,15.802778
9,10,65.548387,19007.396441,48.276242,83.135914,29.961613,54.106408,19.497849,10.026882,17.074839,...,12169.833333,6709.266667,4345.400000,758.500000,356.666667,5.892293e+05,19.589623,17.207435,11.325648,8.043478


In [42]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [43]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume        value
month hour                                    
1     0      38.387045  11.568182   508.308636
      1      38.387045  11.568182   508.308636
      2      38.387045  11.568182   508.308636
      3      38.387045  11.568182   508.308636
      4      38.316935  11.626613   513.657411
...                ...        ...          ...
12    19    121.692500  14.888710   853.341532
      20     65.951855  25.118548  1396.648806
      21     65.951855  25.118548  1396.648806
      22     35.736452  46.086290  1868.078177
      23     35.736452  46.086290  1868.078177

[288 rows x 3 columns]

In [44]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')